In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from datetime import datetime

In [2]:
experiment_datetime = "2023-06-05T17:53:19"
outcomes_df = pd.read_csv('./output/raw/open_exploration/' + experiment_datetime + "_outcomes.csv")
experiments_df = pd.read_csv('./output/raw/open_exploration/' + experiment_datetime + "_experiments.csv")
# TODO: remove index column when you write it to a file, or give it a name

In [3]:
outcomes_df

,Unnamed: 0,A.1 Expected Annual Damage,A.1 Expected Number of Deaths,A.2 Expected Annual Damage,A.2 Expected Number of Deaths,A.4 Expected Annual Damage,A.4 Expected Number of Deaths,A.5 Expected Annual Damage,A.5 Expected Number of Deaths,Total Infrastructure Costs
0,0,2.328738e+07,0.018189,6.817814e+08,0.701015,2.812165e+06,0.001312,0.000000e+00,0.000000,8.460000e+07
1,1,1.921012e+07,0.014007,1.282022e+08,0.131224,0.000000e+00,0.000000,0.000000e+00,0.000000,8.460000e+07
2,2,0.000000e+00,0.000000,3.738095e+07,0.029584,1.391787e+07,0.006031,2.438747e+08,0.179558,8.460000e+07
3,3,0.000000e+00,0.000000,6.387116e+06,0.008742,1.844307e+07,0.012206,0.000000e+00,0.000000,8.460000e+07
4,4,0.000000e+00,0.000000,7.668333e+07,0.071944,4.961275e+07,0.022665,0.000000e+00,0.000000,8.460000e+07
...,...,...,...,...,...,...,...,...,...,...
995,995,3.777971e+06,0.002870,1.253114e+08,0.131248,0.000000e+00,0.000000,0.000000e+00,0.000000,9.566877e+07
996,996,0.000000e+00,0.000000,1.286192e+08,0.118486,0.000000e+00,0.000000,0.000000e+00,0.000000,9.566877e+07
997,997,0.000000e+00,0.000000,2.729749e+07,0.023549,0.000000e+00,0.000000,0.000000e+00,0.000000,9.566877e+07
998,998,7.886415e+07,0.041009,4.201030e+08,0.306517,0.000000e+00,0.000000,0.000000e+00,0.000000,9.566877e+07


## Simple Analysis
Here, we look for statistics about the outcomes of the experiments, separated by policy.

In [4]:
experiments_df.head()

,Unnamed: 0,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,...,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,0,105,84.740803,10.0,0.341607,330.915320,1.0,0.081437,99.946601,1.5,...,0,0,0,0,0,0,0,0,policy 1,dikesnet
1,1,60,317.408704,10.0,0.357019,70.974902,1.5,0.383618,156.311722,10.0,...,0,0,0,0,0,0,0,1,policy 1,dikesnet
2,2,59,41.492248,10.0,0.482082,283.990825,10.0,0.731441,159.979313,1.5,...,0,0,0,0,0,0,0,2,policy 1,dikesnet
3,3,36,76.287353,1.0,0.507364,157.006681,1.5,0.909570,181.330986,1.5,...,0,0,0,0,0,0,0,3,policy 1,dikesnet
4,4,121,189.974602,1.0,0.530336,246.743839,10.0,0.518999,330.866953,1.5,...,0,0,0,0,0,0,0,4,policy 1,dikesnet


In [5]:
# Tag which policy each row belongs to
experiments_df['Policy'] = ''
for idx, row in experiments_df.iterrows():
    if row['0_RfR 0'] == 1:
        if row['A.4_DikeIncrease 0'] == 5:
            experiments_df.at[idx, 'Policy'] = 5
        else:
            experiments_df.at[idx, 'Policy'] = 1
    elif row['3_RfR 0'] == 1:
        experiments_df.at[idx, 'Policy'] = 2
    elif row['A.4_DikeIncrease 0'] == 5:
        experiments_df.at[idx, 'Policy'] = 3
    else:
        experiments_df.at[idx, 'Policy'] = 4

In [6]:
experiments_df = experiments_df.rename(columns={'Unnamed: 0':'Run ID'})
outcomes_df = outcomes_df.rename(columns={'Unnamed: 0':'Run ID'})

In [7]:
policies_df = experiments_df[['Run ID', 'Policy']]
policies_df.head()

,Run ID,Policy
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [8]:
outcomes_df = pd.merge(outcomes_df, policies_df, on='Run ID')
outcomes_df.head()

,Run ID,A.1 Expected Annual Damage,A.1 Expected Number of Deaths,A.2 Expected Annual Damage,A.2 Expected Number of Deaths,A.4 Expected Annual Damage,A.4 Expected Number of Deaths,A.5 Expected Annual Damage,A.5 Expected Number of Deaths,Total Infrastructure Costs,Policy
0,0,2.328738e+07,0.018189,6.817814e+08,0.701015,2.812165e+06,0.001312,0.000000e+00,0.000000,84600000.0,1
1,1,1.921012e+07,0.014007,1.282022e+08,0.131224,0.000000e+00,0.000000,0.000000e+00,0.000000,84600000.0,1
2,2,0.000000e+00,0.000000,3.738095e+07,0.029584,1.391787e+07,0.006031,2.438747e+08,0.179558,84600000.0,1
3,3,0.000000e+00,0.000000,6.387116e+06,0.008742,1.844307e+07,0.012206,0.000000e+00,0.000000,84600000.0,1
4,4,0.000000e+00,0.000000,7.668333e+07,0.071944,4.961275e+07,0.022665,0.000000e+00,0.000000,84600000.0,1


In [11]:
means = outcomes_df.groupby('Policy').mean()
means.head()

,Run ID,A.1 Expected Annual Damage,A.1 Expected Number of Deaths,A.2 Expected Annual Damage,A.2 Expected Number of Deaths,A.4 Expected Annual Damage,A.4 Expected Number of Deaths,A.5 Expected Annual Damage,A.5 Expected Number of Deaths,Total Infrastructure Costs
Policy,,,,,,,,,,
1,99.5,3.297877e+08,0.217339,1.670899e+08,0.158245,2.461893e+07,0.010724,9.336802e+07,0.083400,8.460000e+07
2,299.5,9.344868e+08,0.640564,1.104756e+08,0.104882,1.509666e+07,0.006194,3.487712e+07,0.031995,1.212000e+08
3,499.5,9.344868e+08,0.640564,1.236747e+08,0.117889,2.150025e+06,0.000946,7.803140e+07,0.071101,1.106877e+07
4,699.5,9.344868e+08,0.640564,1.236747e+08,0.117889,0.000000e+00,0.000000,7.902493e+07,0.071944,2.038434e+07
5,899.5,3.297877e+08,0.217339,1.670899e+08,0.158245,2.378009e+06,0.001057,1.065284e+08,0.095730,9.566877e+07
